In [2]:
print lables_arr

[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 

In [3]:
X_train, X_test, y_train, y_test = train_test_split(genotype_data, lables_arr, test_size=0.1,
                                                    random_state=9)

# 随机森林

In [4]:
from sklearn.ensemble import RandomForestClassifier as rf

In [6]:
rf_params = {'n_estimators': 3000, 'criterion': 'entropy', 'max_depth': 3, 'random_state': 1234}
clf = rf(**rf_params)
clf.fit(X_train, y_train)
train_acc = clf.score(X_train, y_train)
test_acc = clf.score(X_test, y_test)

In [7]:
print train_acc, test_acc

1.0 0.54


# 基于一些准则进行特征选取

In [9]:
feature_num = np.size(genotype_data, axis = 1)

## 互信息

In [11]:
from sklearn.metrics import mutual_info_score
MI = np.zeros(feature_num)

for i in range(feature_num):
    MI[i] = mutual_info_score(genotype_data[:, i], lables_arr)
    
plt.figure(3)
plt.plot(MI)
plt.xlabel("feature_index")
plt.ylabel("MI")

mi_rank = np.argsort(np.abs(MI))[-1::-1]
plt.figure(4)
plt.plot(MI[mi_rank])
plt.ylabel("MI")

/usr/local/lib/python2.7/dist-packages/numpy/core/fromnumeric.py:2652: VisibleDeprecationWarning: `rank` is deprecated; use the `ndim` attribute or function instead. To find the rank of a matrix see `numpy.linalg.matrix_rank`.
  VisibleDeprecationWarning)


In [12]:
%pylab

Using matplotlib backend: TkAgg
Populating the interactive namespace from numpy and matplotlib


`%pylab --no-import-all` prevents importing * from pylab and numpy


In [13]:
figure(3)
plot(MI)
xlabel("feature_index")
ylabel("MI")

mi_rank = np.argsort(np.abs(MI))[-1::-1]
figure(4)
plot(MI[mi_rank])
ylabel("MI")

### 随机森林

In [24]:
rf_params['n_estimators'] = 300
clf = rf(**rf_params)
clf.fit(X_train, y_train)
train_acc = clf.score(X_train, y_train)
test_acc = clf.score(X_test, y_test)

max_feature_num = 200
rf_mi_train_acc_list = []
rf_mi_test_acc_list = []
for cur_feature_num in xrange(1, max_feature_num+1):
    feature_ind = mi_rank[0:cur_feature_num]
    cur_X_train = X_train[:, feature_ind]
    cur_X_test = X_test[:, feature_ind]

    clf = rf(**rf_params)
    clf.fit(cur_X_train, y_train)
    mi_train_acc = clf.score(cur_X_train, y_train)
    rf_mi_train_acc_list.append(mi_train_acc)
    mi_test_acc = clf.score(cur_X_test, y_test)
    rf_mi_test_acc_list.append(mi_test_acc)
    
figure(5)
plot(xrange(1, max_feature_num+1), rf_mi_train_acc_list,'r*--')
plot(xrange(1, max_feature_num+1), rf_mi_test_acc_list, 'gs--')

In [25]:
print max(rf_mi_train_acc_list), max(rf_mi_test_acc_list)

0.931111111111 0.81


## GDBT

In [29]:
from sklearn.ensemble import GradientBoostingClassifier
gdbt_params = {'n_estimators': 100, 'max_depth': 3, 'subsample': 0.9, 
          'learning_rate': 0.1, 'min_samples_leaf': 1, 'random_state': 1234}

gdbt_params['n_estimators'] = 200
clf = GradientBoostingClassifier(**gdbt_params)
clf.fit(X_train, y_train)
train_acc = clf.score(X_train, y_train)
test_acc = clf.score(X_test, y_test)

In [30]:
print train_acc, test_acc

1.0 0.5


In [31]:
print clf.oob_improvement_

[  1.74778443e-02  -5.31807421e-03   5.62664042e-03  -1.77072939e-02
  -2.23593087e-04  -1.46819538e-02  -4.49041715e-03  -2.15905122e-03
  -2.50983303e-03  -3.81461862e-03  -2.13095293e-02  -5.09316132e-03
  -1.49554407e-03  -3.41931368e-03  -8.05297243e-03  -5.05803354e-03
  -2.44432467e-03   5.43853578e-03  -9.33651145e-03  -8.83519113e-03
   2.64955923e-03  -7.96523394e-03  -3.62270684e-03  -3.73931188e-03
  -3.60314515e-03  -5.54542383e-03  -8.93556551e-03  -6.03754867e-03
  -3.93879795e-03  -1.10977543e-02  -3.73869410e-03  -1.48007139e-03
  -8.29165107e-04  -8.46396906e-03  -1.47589226e-03  -7.47977930e-03
  -2.70861329e-03   9.14200322e-03  -2.49163268e-03   8.58158344e-04
  -9.93434862e-04  -6.56202294e-03  -3.85469000e-03  -6.99692673e-03
  -2.92091671e-03  -2.65839997e-03  -3.14873706e-03  -2.10946376e-03
  -2.90828659e-03   1.47930902e-04  -6.54456000e-03   2.72118734e-03
  -5.49293237e-03  -2.57654706e-03  -1.34372866e-03  -7.22537271e-03
  -2.16776055e-03  -3.16270643e-03

In [45]:
gdbt_mi_train_acc_list = []
gdbt_mi_test_acc_list = []
gdbt_params['subsample'] = 1
gdbt_params['n_estimators'] = 200
for cur_feature_num in xrange(1, max_feature_num+1):
    feature_ind = mi_rank[0:cur_feature_num]
    cur_X_train = X_train[:, feature_ind]
    cur_X_test = X_test[:, feature_ind]

    clf = GradientBoostingClassifier(**gdbt_params)
    clf.fit(cur_X_train, y_train)
    mi_train_acc = clf.score(cur_X_train, y_train)
    gdbt_mi_train_acc_list.append(mi_train_acc)
    mi_test_acc = clf.score(cur_X_test, y_test)
    gdbt_mi_test_acc_list.append(mi_test_acc)
    
figure(6)
plot(xrange(1, max_feature_num+1), gdbt_mi_train_acc_list,'r*--')
plot(xrange(1, max_feature_num+1), gdbt_mi_test_acc_list, 'gs--')

In [46]:
print max(gdbt_mi_train_acc_list), max(gdbt_mi_test_acc_list)

1.0 0.84


In [47]:
print np.argmax(gdbt_mi_test_acc_list)

103


# 结论
互信息前82名的特征得到的测试集效果最好， 从这82个特征中前向进行剔除特征看效果是否会提升

In [36]:
cur_feature_num = 82
feature_ind = mi_rank[0:cur_feature_num]
cur_X_train = X_train[:, feature_ind]
cur_X_test = X_test[:, feature_ind]
clf = GradientBoostingClassifier(**gdbt_params)
clf.fit(cur_X_train, y_train)
mi_train_acc = clf.score(cur_X_train, y_train)
mi_test_acc = clf.score(cur_X_test, y_test)

In [37]:
print mi_train_acc, mi_test_acc

0.985555555556 0.84


In [38]:
print clf.feature_importances_

[ 0.01899327  0.01758077  0.01133334  0.01559472  0.01832101  0.01341661
  0.01369941  0.01241654  0.01658318  0.00969473  0.00954414  0.01378536
  0.01083615  0.01056841  0.02391773  0.01301598  0.01408381  0.01925907
  0.01468732  0.01350517  0.01610973  0.01467619  0.00691329  0.01249541
  0.01047473  0.01456688  0.01501269  0.01275605  0.01884516  0.00619831
  0.00936878  0.00959829  0.00700818  0.0145365   0.00828712  0.00786879
  0.01498916  0.00855832  0.01540081  0.00721711  0.00743429  0.0098883
  0.01003532  0.00431773  0.00856916  0.01593708  0.01903795  0.0076498
  0.01205824  0.02253112  0.0121903   0.01033782  0.00989067  0.00779281
  0.01445723  0.01041881  0.01314126  0.00726177  0.01495005  0.00776485
  0.01632536  0.01587569  0.01659572  0.0114799   0.01010051  0.01110534
  0.01462398  0.0052181   0.01854182  0.01261601  0.01244281  0.00633086
  0.00938596  0.00810117  0.00937094  0.01380671  0.00316573  0.01130853
  0.02267741  0.00380837  0.01146635  0.00826995]


In [39]:
print feature_ind

[2937 1791 8379 7736   79 8588 9423 1540  961 3753 3587 1592 6793 4525 7841
  756 5943 5936 6840 3771  477 4285 4931 1194 7961  800 8138 1234 3327 2178
 5564 6377 1923 6076 2372 8434 3572 8354 5660 3306 9220 4659 1274 3340 4886
 2201 8587 3009 1598 5224 6970 9277 1226 8389 4919 8383 3816 7741 9195 3397
 5720 7381 5092 4222 6784 7994 9163 6097 1381  679 7414 3926 2258 6603 2982
 7047  592 5368  609 8355 5168 8779]


# 遗传算法

In [78]:
gdbt_params['subsample'] = 1
gdbt_params['n_estimators'] = 200
cur_feature_num = 120
feature_ind = mi_rank[0:cur_feature_num]

popsize = 200 #种群的大小
pop = [np.random.randint(0,2,cur_feature_num).tolist()  for i in range(popsize)]
pc = 0.6 #两个个体交叉的概率
pm = 0.01; #基因突变的概率
results = [[]]
bestindividual = []
bestfit = 0
fitvalue = []
tempop = [[]]

def b2d(b): #将二进制转化为十进制 x∈[0,10]
	t = 0
	for j in range(len(b)):
		t += b[j] * (math.pow(2, j))
	t = t * 10 / 1023
	return t

def best(pop, fitvalue): #找出适应函数值中最大值，和对应的个体
	px = len(pop)
	bestindividual = []
	bestfit = fitvalue[0]
	for i in range(1,px):
		if(fitvalue[i] > bestfit):
			bestfit = fitvalue[i]
			bestindividual = pop[i]
	return [bestindividual, bestfit]

def calfitvalue(objvalue):#转化为适应值，目标函数值越大越好，负值淘汰。
    fitvalue = []
    temp = 0.0
    Cmin = 0;
    for i in range(len(objvalue)):
        if(objvalue[i] + Cmin > 0):
            temp = Cmin + objvalue[i]
        else:
            temp = 0.0
        fitvalue.append(temp)
    return fitvalue

import math

def decodechrom(pop): #将种群的二进制基因转化为十进制（0,1023）
    temp = [];
    for i in range(len(pop)):
        t = 0;
        for j in range(10):
            t += pop[i][j] * (math.pow(2, j))
        temp.append(t)
    return temp

def calobjvalue(pop): #计算目标函数值
    temp1 = [];
    objvalue = [];
    for cur_pop in pop:
        
        if np.size(np.where(np.array(cur_pop)==1)[0])<1:
            objvalue.append(0)
            continue
        
 #       print np.size(np.where(cur_pop==1)[0])
        cur_feature_indices = feature_ind[np.where(np.array(cur_pop)==1)[0]]
        cur_X_train = X_train[:, cur_feature_indices]
        cur_X_test = X_test[:, cur_feature_indices]
        clf = GradientBoostingClassifier(**gdbt_params)
        clf.fit(cur_X_train, y_train)
 #       mi_train_acc = clf.score(cur_X_train, y_train)
        mi_test_acc = clf.score(cur_X_test, y_test)
        objvalue.append(mi_test_acc)
    return objvalue #目标函数值objvalue[m] 与个体基因 pop[m] 对应 

import random

def crossover(pop, pc): #个体间交叉，实现基因交换
    poplen = len(pop)
    for i in range(poplen - 1):
        if(random.random() < pc):
            cpoint = random.randint(0,len(pop[0]))
            temp1 = []
            temp2 = []
            temp1.extend(pop[i][0 : cpoint])
            temp1.extend(pop[i+1][cpoint : len(pop[i])])
            temp2.extend(pop[i+1][0 : cpoint])
            temp2.extend(pop[i][cpoint : len(pop[i])])
            pop[i] = temp1
            pop[i+1] = temp2

import random

def mutation(pop, pm): #基因突变
    px = len(pop)
    py = len(pop[0])
    
    for i in range(px):
        if(random.random() < pm):
            mpoint = random.randint(0,py-1)
            if(pop[i][mpoint] == 1):
                pop[i][mpoint] = 0
            else:
                pop[i][mpoint] = 1
                
import random

def sum(fitvalue):
    total = 0
    for i in range(len(fitvalue)):
        total += fitvalue[i]
    return total

def cumsum(fitvalue):
    for i in range(len(fitvalue)):
        t = 0;
        j = 0;
        while(j <= i):
            t += fitvalue[j]
            j = j + 1
        fitvalue[i] = t;

def selection(pop, fitvalue): #自然选择（轮盘赌算法）
	newfitvalue = []
	totalfit = sum(fitvalue)
	if totalfit==0.0:
		pop = [np.random.randint(0,2,cur_feature_num).tolist()  for i in range(popsize)]
        return

	for i in range(len(fitvalue)):
		newfitvalue.append(fitvalue[i] / totalfit)
	cumsum(newfitvalue)
	ms = [];
	poplen = len(pop)
	for i in range(poplen):
		ms.append(random.random()) #random float list ms
	ms.sort()
	fitin = 0
	newin = 0
	newpop = pop
	while newin < poplen:
		if(ms[newin] < newfitvalue[fitin]):
			newpop[newin] = pop[fitin]
			newin = newin + 1
		else:
			fitin = fitin + 1
	pop = newpop

for i in range(100): #繁殖100代
    print i
    objvalue = calobjvalue(pop) #计算目标函数值
    fitvalue = calfitvalue(objvalue); #计算个体的适应值
    [bestindividual, bestfit] = best(pop, fitvalue) #选出最好的个体和最好的函数值
    results.append([bestfit,bestindividual]) #每次繁殖，将最好的结果记录下来
    selection(pop, fitvalue) #自然选择，淘汰掉一部分适应性低的个体
    crossover(pop, pc) #交叉繁殖
    mutation(pop, pc) #基因突变

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99


In [80]:
print results

[[], [0.84999999999999998, [0, 0, 1, 1, 1, 1, 0, 1, 0, 1, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 1, 1, 0, 0, 1, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 1, 1, 1, 0, 0, 0, 1, 1, 1, 0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 0, 0, 1, 1, 1, 0, 0, 1, 1, 1, 0, 0, 1, 1, 1, 0, 1, 1, 0, 0, 1, 1, 0, 1, 0, 0, 0]], [0.84999999999999998, [0, 0, 1, 1, 1, 1, 0, 1, 0, 1, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 1, 1, 0, 0, 1, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 1, 1, 1, 0, 0, 0, 1, 1, 1, 0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 0, 0, 1, 1, 1, 0, 0, 1, 1, 1, 0, 0, 1, 1, 1, 0, 1, 1, 0, 0, 1, 1, 0, 1, 0, 0, 0]], [0.81000000000000005, [0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0, 1, 1, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1

In [82]:
print sum(results[-6][1])

72


In [86]:
max_acc = 0
for cur_result in results:
    if len(cur_result)==0:
        continue
    cur_acc = cur_result[0]
    if cur_acc>max_acc:
        max_acc = cur_acc
        feature_flag = cur_result[1]
        
best_feature = feature_ind[np.where(np.array(feature_flag)==1)[0]]
print max_acc
print best_feature
print len(best_feature)

0.86
[1791 8379   79 8588 1540  961 3753 3587 4525 5936 7961 8138 1234 6377 1923
 3572 3306 3340 4886 5224 6970 9277 1226 3816 3397 5720 7381 6784 9163 6097
 1381 7414 3926 2982  592 5368 8355 1974 5904 2091 4254 6759 7622 8796 3384
 7302 7672 4049 8995 5157 7935 5273 7365 3077 5906]
55


In [90]:
print MI[best_feature]

[ 0.01009204  0.0084789   0.00790148  0.00781446  0.00730066  0.00720742
  0.00708501  0.00678139  0.00647837  0.00645415  0.00597278  0.0057852
  0.0056969   0.0055131   0.00549771  0.00536705  0.00527317  0.00517987
  0.00515105  0.0049785   0.00495251  0.00493022  0.00490166  0.00485899
  0.00480929  0.00480081  0.00477613  0.00475673  0.00473247  0.00473203
  0.00471263  0.0046856   0.00468038  0.00465749  0.00463503  0.00462565
  0.00458339  0.00451413  0.00449469  0.00447955  0.00447709  0.00445471
  0.00443459  0.00443322  0.00441209  0.00440781  0.0043974   0.00439115
  0.0043658   0.00436335  0.00432519  0.00432203  0.0043216   0.00431365
  0.00430654]


In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.cross_validation import train_test_split
from sklearn.cross_validation import KFold
from multiprocessing import Pool

genotype = pd.read_csv("/home/flame/Desktop/solutions/data/genotype.csv")
genotype_data = genotype.values

lables_list = []
with open("/home/flame/Desktop/solutions/data/phenotype.txt", mode = "r") as f:
    for line in f.readlines():
        lables_list.append(int(line.strip()))
        
lables_arr = np.array(lables_list)

In [110]:
gdbt_params['subsample'] = 1
gdbt_params['n_estimators'] = 200
text_tree_depth_result = {}
for i in xrange(3,9):
    gdbt_params['max_depth'] = i
    cur_X_train = X_train[:, best_feature]
    cur_X_test = X_test[:, best_feature]
    clf = GradientBoostingClassifier(**gdbt_params)
    clf.fit(cur_X_train, y_train)
    mi_train_acc = clf.score(cur_X_train, y_train)
    mi_test_acc = clf.score(cur_X_test, y_test)
    text_tree_depth_result[i] = (mi_train_acc, mi_test_acc)    

In [111]:
print text_tree_depth_result

{3: (0.96111111111111114, 0.85999999999999999), 4: (0.99888888888888894, 0.85999999999999999), 5: (1.0, 0.81999999999999995), 6: (1.0, 0.83999999999999997), 7: (1.0, 0.77000000000000002), 8: (1.0, 0.75)}


### 进一步统计每个位点各个取值下得病的百分比

In [109]:
def disease_statistic(cur_feature_ind):
    cur_feature = genotype_data[:,cur_feature_ind]
    disease_dict = {}
    for i in xrange(1,4):
        ind_arr1 = np.where(cur_feature==i)[0]
        positive_num = len(np.where(lables_arr[ind_arr1]==1)[0])
        negtive_num = len(ind_arr1)-positive_num
        disease_dict[i] = [positive_num, negtive_num]
    return disease_dict
 
def disease_statistic_plot(disease_dict):
    men1 = []
    men2 = []
    for i in xrange(1,4):
        cur_item = disease_dict[i]
        men2.append(cur_item[0])
        men1.append(sum(cur_item))
        
    men1 = tuple(men1)
    men2 = tuple(men2)
    
    ind = np.arange(3) + .15
    width = 0.40
    fig, ax = plt.subplots()
    rects1 = ax.bar(ind, men1, width, color='g') 
    rects2 = ax.bar(ind, men2, width, color='r')
    
    ax.set_xticks(ind+width-0.2)
    ax.set_xticklabels(('1', '2', '3'))
    plt.show()
disease_dict = disease_statistic(1)
disease_statistic_plot(disease_dict)

# 找出与疾病最相关的一个或者几个基因

In [87]:
import json
f = open("/home/flame/Desktop/solutions/data/gene_encoding.json",mode="r")
gene_encoding = json.load(f)
f.close()
print gene_encoding

{u'216': [6723, 6724, 6725, 6726, 6727, 6728, 6729, 6730, 6731, 6732, 6733, 6734, 6735, 6736, 6737, 6738, 6739, 6740, 6741, 6742, 6743, 6744, 6745, 6746, 6747, 6748, 6749, 6750, 6751, 6752, 6753, 6754, 6755, 6756, 6757, 6758, 6759, 6760, 6761, 6762, 6763, 6764, 6765, 6766, 6767, 6768, 6769, 6770, 6771, 6772, 6773, 6774, 6775, 6776, 6777], u'217': [6778, 6779, 6780, 6781, 6782, 6783, 6784, 6785, 6786, 6787, 6788, 6789, 6790, 6791, 6792, 6793, 6794, 6795, 6796, 6797], u'214': [6663, 6664, 6665, 6666, 6667, 6668, 6669, 6670, 6671, 6672, 6673, 6674, 6675, 6676, 6677, 6678, 6679, 6680, 6681, 6682, 6683, 6684, 6685, 6686, 6687, 6688, 6689, 6690, 6691, 6692, 6693, 6694, 6695, 6696, 6697, 6698, 6699, 6700, 6701], u'215': [6702, 6703, 6704, 6705, 6706, 6707, 6708, 6709, 6710, 6711, 6712, 6713, 6714, 6715, 6716, 6717, 6718, 6719, 6720, 6721, 6722], u'212': [6632, 6633, 6634, 6635, 6636, 6637, 6638, 6639, 6640, 6641, 6642, 6643, 6644, 6645, 6646, 6647, 6648, 6649], u'213': [6650, 6651, 6652, 6653

In [1]:
target_gene = {}
for cur_feature in best_feature:
    for cur_gene, cur_snp in gene_encoding.iteritems():
        if cur_feature in cur_snp:
            if target_gene.has_key(cur_gene):
                target_gene[cur_gene].append(cur_feature)
            else:
                target_gene[cur_gene] = [cur_feature]

sorted(target_gene.iteritems(), key=lambda d:d[0])
print target_gene

NameError: name 'best_feature' is not defined

In [ ]:
gdbt_params['subsample'] = 1
gdbt_params['n_estimators'] = 200
